In [1]:
import pandas as pd
import time
import tweepy
import configparser
import datetime
import pymysql
from sqlalchemy import create_engine
import mysql.connector

# Twitter retrieval search

In [2]:
def mysql_conf():
        config = configparser.ConfigParser()
        config.read('config.ini')
        username = config['mysql']['username']
        password = config['mysql']['password']
        hostname = config['mysql']['hostname']
        database = config['mysql']['database']
        return username,password,hostname,database
    
def create_database():
    username,password,hostname,database = mysql_conf()
    conn = mysql.connector.connect(host = hostname,user = username,password = password)
    cursor = conn.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS twitter")
    cursor.close()


def create_table():
    username,password,hostname,database = mysql_conf()
    conn = mysql.connector.connect(host = hostname,user = username,password = password, database = database)
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS datamined(tweet_id double NOT NULL,\
                            name varchar(255) NOT NULL,\
                            screen_name varchar(255) NOT NULL,\
                            retweet_count int NOT NULL,\
                            text text(65535) NOT NULL,\
                            mined_at Timestamp NOT NULL,\
                            created_at Timestamp NOT NULL,\
                            favourite_count int NOT NULL,\
                            hashtags text(65535),\
                            status_count int NOT NULL,\
                            followers_count int NOT NULL,\
                            location text(65535),\
                            source_device varchar(255) NOT NULL,\
                            processed boolean NOT NULL, PRIMARY KEY (tweet_id))")
    cursor.close()

def check_record(value):
    username,password,hostname,database = mysql_conf()
    conn = mysql.connector.connect(host = hostname,user = username,password = password, database = database)
    cursor = conn.cursor()

    quary = """SELECT tweet_id, COUNT(*) FROM datamined WHERE tweet_id =%s GROUP BY tweet_id"""
    cursor.execute(quary,(value,))
    row_count1 = cursor.fetchall()
    row_count = cursor.rowcount
    cursor.close()

    return row_count

def check_redundancy(df):
    
    for index in df.index:
        if check_record(int(df["tweet_id"][index])) == 1:
            print(df["tweet_id"][index],"is redundant, deleteing record from dataframe!")
            df = df.drop(index)
    return df

def write_to_mysql(df):
    print("........Storing in Mysql Database", len(df))
    username,password,hostname,database = mysql_conf()


    df.hashtags = df.hashtags.astype(str)
    df["processed"] = False

    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=database, user=username, pw=password))
    df.to_sql('datamined', con = engine, if_exists = 'append',index = False, chunksize = 1000)
    
    print("................................................ Resetting Dataframe")
    tweets = get_df()
    return tweets

In [3]:
class Linstener(tweepy.Stream):
    tweets_df = []
    limit = 100
    counter = 0

    def on_status(self, status):
        self.counter +=1
        print("...... Start streaming from Twitter!Retrieve tweets from twitter:",self.counter,end="\r")
        self.tweets_df.append(status)
        if len(self.tweets_df) == self.limit:
            self.disconnect()
            
def stream_authentication(keywords):
    config = configparser.ConfigParser()
    config.read('config.ini')

    api_key = config['twitter']['api_key']
    api_key_secret = config['twitter']['api_key_secret']

    access_token = config['twitter']['access_token']
    access_token_secret = config['twitter']['access_token_secret']

    # authenticate
    auth = tweepy.OAuthHandler(api_key, api_key_secret)
    auth.set_access_token(access_token, access_token_secret)

    # authenticate
    stream_tweet = Linstener(api_key, api_key_secret, access_token, access_token_secret)
    stream_tweet.filter(track=keywords)
    return stream_tweet

def get_df():
    return pd.DataFrame(
    columns = [
            "tweet_id",
            "name",
            "screen_name",
            "retweet_count",
            "text",
            "mined_at",
            "created_at",
            "favourite_count",
            "hashtags",
            "status_count",
            "followers_count",
            "location",
            "source_device"])

def get_df_tweet(data):
    return pd.DataFrame(data,
    columns = [
            "tweet_id",
            "name",
            "screen_name",
            "retweet_count",
            "text",
            "mined_at",
            "created_at",
            "favourite_count",
            "hashtags",
            "status_count",
            "followers_count",
            "location",
            "source_device"])
    
def process_tweets(public_tweets):
    data = []
    
    for tweet in public_tweets.tweets_df:
        if tweet.truncated == False:
            data.append([tweet.id,
                         tweet.user.name,
                         tweet.user.screen_name,
                         tweet.retweet_count,
                         tweet.text,
                         datetime.datetime.now(),
                         tweet.created_at,
                         tweet.favorite_count,
                         tweet.entities["hashtags"],
                         tweet.user.statuses_count,
                         tweet.user.followers_count,
                         tweet.place,
                         tweet.source])
        else:
            data.append([tweet.id,
                         tweet.user.name,
                         tweet.user.screen_name,
                         tweet.retweet_count,
                         tweet.extended_tweet['full_text'],
                         datetime.datetime.now(),
                         tweet.created_at,
                         tweet.favorite_count,
                         tweet.entities["hashtags"],
                         tweet.user.statuses_count,
                         tweet.user.followers_count,
                         tweet.place,
                         tweet.source])
        
    df = get_df_tweet(data)
    return df

In [8]:
def scrap_twitter(keywords=['']):
    connection_num = 1
    create_database()
    create_table()
    
    while True:
        # authenticate
        if Linstener.tweets_df:
            Linstener.tweets_df = []
        print(" Connection attempted:",connection_num,",retrieved time:",datetime.datetime.now())
        connection_num += 1
#         stream_tweet = stream_authentication(keywords)
#         df = process_tweets(stream_tweet)
#         df = check_redundancy(df)
#         df = write_to_mysql(df)
        write_to_mysql(check_redundancy(process_tweets(stream_authentication(keywords))))
        Linstener.tweets_df = []
        print("........................................... Sleeping for 10 seconds!")
        time.sleep(10)

In [9]:
keywords = ['#BTC']
stream_tweet = scrap_twitter(keywords)

 Connection attempted: 1 ,retrieved time: 2022-04-04 00:09:26.215239


Stream connection closed by Twitter


........Storing in Mysql Database 100etrieve tweets from twitter: 100
................................................ Resetting Dataframe
........................................... Sleeping for 10 seconds!
 Connection attempted: 2 ,retrieved time: 2022-04-04 00:10:29.891987


Stream connection closed by Twitter


........Storing in Mysql Database 100etrieve tweets from twitter: 100
................................................ Resetting Dataframe
........................................... Sleeping for 10 seconds!
 Connection attempted: 3 ,retrieved time: 2022-04-04 00:11:41.637557


...

Stream connection closed by Twitter


........Storing in Mysql Database 100etrieve tweets from twitter: 100
................................................ Resetting Dataframe
........................................... Sleeping for 10 seconds!
 Connection attempted: 7 ,retrieved time: 2022-04-04 00:18:03.502413


Stream connection closed by Twitter


........Storing in Mysql Database 100etrieve tweets from twitter: 100


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1.510773672168018e18' for key 'datamined.PRIMARY'")
[SQL: INSERT INTO datamined (tweet_id, name, screen_name, retweet_count, text, mined_at, created_at, favourite_count, hashtags, status_count, followers_count, location, source_device, processed) VALUES (%(tweet_id)s, %(name)s, %(screen_name)s, %(retweet_count)s, %(text)s, %(mined_at)s, %(created_at)s, %(favourite_count)s, %(hashtags)s, %(status_count)s, %(followers_count)s, %(location)s, %(source_device)s, %(processed)s)]
[parameters: ({'tweet_id': 1510773638311645190, 'name': 'Meek Nike', 'screen_name': 'Meeknike88', 'retweet_count': 0, 'text': '@elvis_eth @OpenSeaGeek ✅You can buy now\n#BTCbullDog 💥🚀\nThe most transparent project 2022\n#KYC\n🤑💥10% rewards in #BTC\n💥Trading tools\n 💥Dual  Staking System with High APR\n💥P2E Games\n💥🚀Dappe: https://t.co/eMk2S9INru\n💥✅🤑pinksale:\nhttps://t.co/GYjJSdMfgS', 'mined_at': datetime.datetime(2022, 4, 4, 0, 19, 4, 397198), 'c